In [1]:
# 지도 라이브러리 Folium
! pip install folium 

# 드래그 앤 드롭, 파일 선택 구현
!pip install tkinterdnd2

# 지도
import folium
# 지도에 이미지 넣어주는 base, iframe
import base64
from folium import IFrame

#PIL로 이미지 크기 줄였을 때 이미지 돌아가는 오류 제거 io, imageops
import io
from PIL import Image, ImageOps

# exif 정보 가져오는 라이브러리
from PIL.ExifTags import TAGS

# 드래그 앤 드롭 구현
import tkinter as tk
from tkinterdnd2 import DND_FILES, TkinterDnD
import os
from tkinter import filedialog

import math

In [2]:
#드래그 앤 드롭
def drop(event):
    file_paths = event.data
    file_paths_str = ''.join(file_paths)
    file_paths_list = file_paths_str.split('\n')
    for file_path in file_paths_list:
        file_name = os.path.basename(file_path)
        file_name = file_name.replace("\n", "")
        file_listbox.insert(tk.END, file_name)
        image.append(file_path)

def open_file_dialog():
    file_paths = filedialog.askopenfilenames()
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        file_name = file_name.replace("\n", "")
        file_listbox.insert(tk.END, file_name)
        image.append(file_path)

# Tkinter 윈도우 생성
window = TkinterDnD.Tk()

# 드롭 영역 설정
drop_frame = tk.Frame(window, width=400, height=400)
drop_frame.pack()
drop_frame.drop_target_register(DND_FILES)
drop_frame.dnd_bind('<<Drop>>', drop)

# 파일 목록을 표시할 리스트박스 생성
file_listbox = tk.Listbox(window, width=80)
file_listbox.pack()

# 스크롤바 생성
scrollbar = tk.Scrollbar(window)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
file_listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=file_listbox.yview)

# 파일 선택 버튼 생성
button = tk.Button(window, text='파일 선택', command=open_file_dialog)
button.pack()

# 이미지 파일 경로를 저장할 리스트
image = []

# 윈도우 실행
window.mainloop()


In [3]:
gps_lat = []
gps_lon = []
popup_title = []

# 이미지의 exif (날짜, GPS값) 가져오는 함수
def exif(num):
    image_exif = Image.open(image[num])
    info = image_exif._getexif()
    image_exif.close()

    # 새로운 딕셔너리 생성
    taglabel = {}

    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        taglabel[decoded] = value

    
    exifGPS = taglabel['GPSInfo']
    latData = exifGPS[2]
    lonData = exifGPS[4]

    
    # 도, 분, 초 계산
    latDeg = latData[0]
    latMin = latData[1]
    latSec = latData[2]

    lonDeg = lonData[0]
    lonMin = lonData[1]
    lonSec = lonData[2]

    # 도 decimal로 나타내기
    # 위도 계산
    Lat = (latDeg + (latMin + latSec / 60.0) / 60.0)
    # 북위, 남위인지를 판단, 남위일 경우 -로 변경
    if exifGPS[1] == 'S':
        Lat = Lat * -1

    # 경도 계산
    Lon = (lonDeg + (lonMin + lonSec / 60.0) / 60.0)
    # 동경, 서경인지를 판단, 서경일 경우 -로 변경
    if exifGPS[3] == 'W':
        Lon = Lon * -1

    # 위도 경도 저장
    gps_lat.append(Lat)
    gps_lon.append(Lon)

    # 날짜 저장
    exifDATE = taglabel['DateTime']
    popup_title.append(exifDATE)

    
# exif 정보 추출하기
for num in range(len(image)):
    try:
        exif(num)
    except:
        print("exif 메타정보가 없습니다.")

In [4]:
# 실행시 중앙 값
m = folium.Map(location=[33, 120], zoom_start=2)

# 지도 표현하기

for i in range(len(gps_lat)):
    #이미지 사이즈 줄이기 경로
    image_resize_PATH = image[i].split('.')[0]+'_'+popup_title[i].replace(':','.')+'.'+image[i].split('.')[1]

    #이미지 사이즈 줄이기
    img = Image.open(image[i]) # image 배열을 사용해줘야함 # exif 함수도 작성해야함
    
    
    # 이미지 사이즈 줄였을 때  이미지가 돌아가는 PIL 오류 수정
    img_resize = img.resize((256, 256))
    fixed_image = ImageOps.exif_transpose(img_resize)
    bytes_io = io.BytesIO()
    
    # 저장하기
    fixed_image.save(image_resize_PATH)

    # 핀 만들고, popup, 이미지 넣어주기
    encoded = base64.b64encode(open(image_resize_PATH, 'rb').read())
    html = '<img src="data:image/png;base64,{}">'.format
    iframe = IFrame(html(encoded.decode('UTF-8')), width=256 + 30, height=256 + 30) # + 30은 스크롤바 안생기게 하기 위해 키워준 것
    popup = folium.Popup(iframe, max_width=256 + 30)
    folium.Marker([gps_lat[i], gps_lon[i]], tooltip=popup_title[i], popup = popup ,icon=folium.Icon(color="red")).add_to(m)
    


In [5]:
m